In [1]:
!hostname

v026.ib.bridges2.psc.edu


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import Dropout
import tensorflow_addons as tfa
from qhoptim.tf import QHAdamOptimizer
from tensorflow.keras import callbacks
import keras_tuner as kt
import os
import random
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint
import logging

2024-03-16 15:26:54.890891: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-16 15:26:55.083963: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2024-03-16 15:26:55.993167: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/packages/cuda/v11.7.1/lib64:/opt/packages/cuda/v11.7.1/nvvm/lib64:/opt/packages/cuda/v11.7.1/extras/CUPTI/lib64:/jet/home/jlin96/.conda/envs/wandbenv/lib/
2024-03-16 15:26:55.993292: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/packages/cuda/v11.7.1/lib64:/opt/packages/cuda/v11.7.1/nvvm/lib64:/opt/packages/cuda/v11.7.1/extras/CUPTI/lib64:/jet/home/jlin96/.conda/envs/wandbenv/lib/
2024-03-16 15:26:55.993301: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU wit

In [3]:
####################
# uncomment below for debugging
####################

# os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
# random.seed(hash('setting random seeds') % 2**32 - 1)
# np.random.seed(hash('improves reproducibility') % 2**32 - 1)
# tf.random.set_seed(hash('by removing stochasticity') % 2**32 - 1)
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     try:
#         # Currently, memory growth needs to be the same across GPUs
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)
#     except RuntimeError as e:
#         # Memory growth must be set before GPUs have been initialized
#         print(e)

In [4]:
data_path = '/ocean/projects/atm200007p/jlin96/nnspreadtesting_2/bigdata/training/training_data/'
data_path = '/ocean/projects/atm200007p/jlin96/nnspreadtesting_2/overfitmepls/training/training_data/'

# data_path  = '/dev/shm/'
train_input = np.load(data_path + 'train_input.npy', mmap_mode='r')
train_target = np.load(data_path + 'train_target.npy', mmap_mode='r')
val_input = np.load(data_path + 'val_input.npy', mmap_mode='r')
val_target = np.load(data_path + 'val_target.npy', mmap_mode='r')

batch_size = 5000
shuffle_buffer = 100000

def data_generator(inputs, targets):
    # Iterate over the dataset in batches
    for i in range(0, len(inputs)):
        # Yield a batch of data
        yield inputs[i], targets[i]

with tf.device('/CPU:0'):
    train_ds = tf.data.Dataset.from_generator(
        lambda: data_generator(train_input, train_target),
                               output_signature=(tf.TensorSpec(shape=(175), dtype=tf.float32),
                                                 tf.TensorSpec(shape=(55), dtype=tf.float32))).shuffle(buffer_size=shuffle_buffer).batch(batch_size)

    val_ds = tf.data.Dataset.from_generator(
        lambda: data_generator(val_input, val_target),
                               output_signature=(tf.TensorSpec(shape=(175), dtype=tf.float32),
                                                 tf.TensorSpec(shape=(55), dtype=tf.float32))).shuffle(buffer_size=shuffle_buffer).batch(batch_size)

train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

2024-03-16 15:20:43.300715: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-16 15:20:43.824901: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14745 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:b2:00.0, compute capability: 7.0


In [5]:
!wandb login

wandb: Currently logged in as: jerrylin247365 (cbrain). Use `wandb login --relogin` to force relogin


In [6]:
def build_model(hp:dict):
    alpha = hp["leak"]
    dp_rate = hp["dropout"]
    batch_norm = hp["batch_normalization"]
    model = Sequential()
    hiddenUnits = hp['hidden_units']
    model.add(Dense(units = hiddenUnits, input_dim=175, kernel_initializer='normal'))
    model.add(LeakyReLU(alpha = alpha))
    if batch_norm:
        model.add(BatchNormalization())
    model.add(Dropout(dp_rate))
    for i in range(hp["num_layers"]):
        model.add(Dense(units = hiddenUnits, kernel_initializer='normal'))
        model.add(LeakyReLU(alpha = alpha))
        if batch_norm:
            model.add(BatchNormalization())
        model.add(Dropout(dp_rate))
    model.add(Dense(55, kernel_initializer='normal', activation='linear'))
    initial_learning_rate = hp["lr"]
    optimizer = hp["optimizer"]
    if optimizer == "adam":
        optimizer = keras.optimizers.Adam(learning_rate = initial_learning_rate)
    elif optimizer == "RMSprop":
        optimizer = keras.optimizers.RMSprop(learning_rate = initial_learning_rate)
    elif optimizer == "RAdam":
        optimizer = tfa.optimizers.RectifiedAdam(learning_rate = initial_learning_rate)
    elif optimizer == "QHAdam":
        optimizer = QHAdamOptimizer(learning_rate = initial_learning_rate, nu2=1.0, beta1=0.995, beta2=0.999)
    model.compile(optimizer = optimizer, loss = 'mse', metrics = ["mse"])
    return model

In [7]:
num_epochs = 5
hp = {'leak':0.37343, \
      'dropout':0.042309, \
      'lr':0.0011942, \
      'hidden_units':336, \
      'num_layers':6, \
      'optimizer':"RAdam", \
      'batch_normalization':1}
model = build_model(hp)

In [8]:
wandb.init(
    # set the wandb project where this run will be logged
    project="debug221", 
    # track hyperparameters and run metadata
    config=hp,
    name='newstandard'
)

wandb: Currently logged in as: jerrylin247365 (cbrain). Use `wandb login --relogin` to force relogin


In [9]:
model.fit(train_ds, validation_data = val_ds, epochs = num_epochs, batch_size = batch_size, callbacks = [WandbMetricsLogger(), WandbModelCheckpoint('tuning_directory')])

Epoch 1/5
    749/Unknown - 335s 430ms/step - loss: 0.4767 - mse: 0.4767

KeyboardInterrupt: 

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job
